In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from random import randint
import socket
import socks

In [ ]:
def get_job_title(soup): 
    for i in soup.find_all(name="div", attrs={'class':"css-17x2pwl e11nt52q5"}):
        job = i.text
    return job

def get_company(soup): 
    for i in soup.find_all(name="div", attrs={'class':"css-16nw49e e11nt52q1"}):
        comp = i.text.split('.css')[0]
    return comp

def get_location(soup): 
    for i in soup.find_all(name="div", attrs={'class':"css-13et3b1 e11nt52q2"}):
        loc = i.text
    return loc

def get_rating(soup):
    for i in soup.find_all(name="span", attrs={'class':"css-1pmc6te e11nt52q3"}):
        rating = i.text.split('.css')[0]
    return rating

def get_links(soup):
    links = []
    for i in soup.find_all(name="a", attrs={"target":"_blank"}):
        for j in i:
            href = i.get('href')
            if '/partner/jobListing.htm' in href:
                links.append(href)
    return list(set(links))

def get_soup(url):
    page = requests.get(url)#,headers = {'User-agent':'Mozilla/5.0 (compatible; ABrowse 0.4; Syllable)'})
    if page.status_code == 200:
        soup = BeautifulSoup(page.text, "html.parser")
    else:
        print('Status code is not 200')
        soup = 0
        time.sleep(120)
    return soup

def get_job_descr(soup):
    text=''
    for i in soup.find_all(name="div", attrs={"class":"css-jfggi0 e1eh6fgm2"}):
        for j in i.find_all(name="div", attrs={"class":"desc css-58vpdc ecgq1xb3"}):
            text += ' ' + (str(j.text))
    return text


In [ ]:
# socks.set_default_proxy(socks.SOCKS5,'localhost',9150)
# socket.socket=socks.socksocket

data = pd.DataFrame(columns=['title', 'company_rating', 
                             'company', 'location', 'description', 'url'])
link = 'https://www.glassdoor.com/Job/tel-aviv-yafo-data-scientist-jobs-SRCH_IL.0,13_IC2421096_KO14,28.htm?sortBy=date_desc'

soup = get_soup(link)
links = get_links(soup)
df = pd.DataFrame(columns=data.columns)
df['url'] = links
for i,l in enumerate(links):
    link = 'https://www.glassdoor.com' +l
#     df.loc[i]['url'] = link
    print(link)
    page = get_soup(link)
    df.loc[i]['description'] = get_job_descr(page)
    df.loc[i]['company_rating'] = get_rating(page)
    df.loc[i]['title'] = get_job_title(page)
    df.loc[i]['company'] = get_company(page)
    df.loc[i]['location'] = get_location(page)
    time.sleep(randint(3,5))


In [ ]:
data.shape

In [ ]:
data.dropna(inplace=True)
data.drop_duplicates(inplace=True)
data['description'] = data.description.map(lambda x: clear(x))
data.reset_index(inplace=True, drop = True)

In [ ]:
data.to_csv('new_g.csv',index=False)